## Imports

In [1]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=c02e53668145bf44f623a56b21fc732446bdf0cfd2415b2002e493bf094d2e4c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
data_drive = "drive/MyDrive/ebay_data_2023"
os.listdir(data_drive)

['Train_Tagged_Titles.tsv',
 'Listing_Titles.tsv',
 'train_data_tokenized.csv',
 'train_data_ffilled.csv',
 '.DS_Store',
 'train_dfBI.csv',
 'models',
 'results']

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
!pip install distance
import distance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=1c894a5b85b827fe866deb39a7098768e841b06d8684cdcff3e3de5231e7bd32
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [7]:
import pandas as pd
import numpy as np
import csv

In [8]:
from transformers import AutoTokenizer
import datasets
from transformers import DataCollatorForTokenClassification
import evaluate
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from torch.optim import AdamW
from transformers import get_scheduler
from tensorflow.python.client import device_lib

## Preprocess Data

In [9]:
train_df = pd.read_csv(data_drive + "/Train_Tagged_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
old_tag = train_df.Tag.copy()
train_df

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
55178,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe,Produktart
55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Wanderschuh,NaN
55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag
55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag


In [10]:
#quiz dataset
test_records = pd.read_csv(data_drive + "/Listing_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE).loc[5000:29999,:].reset_index(drop=True)


In [11]:
aspect_names = set(pd.unique(old_tag)) - set([np.nan])
aspect_names, len(aspect_names) # includes "No Tag", "Obscure"

({'Abteilung',
  'Aktivität',
  'Akzente',
  'Anlass',
  'Besonderheiten',
  'Charakter',
  'Charakter Familie',
  'Dämpfungsgrad',
  'EU-Schuhgröße',
  'Erscheinungsjahr',
  'Farbe',
  'Futtermaterial',
  'Gewebeart',
  'Herstellernummer',
  'Herstellungsland und -region',
  'Innensohlenmaterial',
  'Jahreszeit',
  'Laufsohlenmaterial',
  'Marke',
  'Maßeinheit',
  'Modell',
  'Muster',
  'No Tag',
  'Obermaterial',
  'Obscure',
  'Produktart',
  'Produktlinie',
  'Schuhschaft-Typ',
  'Schuhweite',
  'Stil',
  'Stollentyp',
  'Thema',
  'UK-Schuhgröße',
  'US-Schuhgröße',
  'Verschluss',
  'Zwischensohlen-Typ'},
 36)

In [12]:
train_df.Tag = train_df.Tag.fillna("I-" + train_df.Tag.ffill()[train_df.Tag.isna()])
train_df.Tag.loc[train_df.Tag == "I-No Tag"] = "No Tag" # correct for the No Tag (there can be no interior)
train_df.Tag.loc[train_df.Tag == "I-Obscure"] = "Obscure" # correct for the No Tag (there can be no interior)

In [13]:
# train_df.to_csv(data_drive + "/train_dfBI.csv")

In [14]:
pd.unique(train_df.Tag), len(pd.unique(train_df.Tag)) # notice this does not encompass all 70 classes (36*2) - 2 (for No Tag, Obscure)
pass

In [15]:
not_to_include = set(["No Tag", "Obscure"])
asp_arr = np.array(list(aspect_names - not_to_include))
asp_arr.sort()
interior_arr = ("I-"+ pd.Series(list(aspect_names - not_to_include))).sort_values().values
all_aspect_names = np.append(asp_arr, interior_arr)
all_aspect_names = np.append(all_aspect_names, list(not_to_include))
#all aspect names, all interior aspect names, Obscure, No Tag = 70 aspect names

In [16]:
id2label = dict(zip(list(range(len(all_aspect_names))), all_aspect_names))
label2id = dict(zip(all_aspect_names, list(range(len(all_aspect_names)))))

In [17]:
train_df["ner_tags"] = train_df.Tag.apply(lambda val: label2id[val])
train_df["Record Number"] = train_df["Record Number"].astype(int)

## Tokenize

The next step is to load a DistilBERT tokenizer to preprocess the `tokens` field:

In [18]:
cased_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased", do_lower_case=False)
lowercased_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

In [19]:
tokenized_input = lowercased_tokenizer("Here is MY sentence")
tokens = lowercased_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]', 'here', 'is', 'my', 'sentence', '[SEP]']

In [20]:
train_ds = datasets.Dataset.from_pandas(train_df)
train_ds

Dataset({
    features: ['Record Number', 'Title', 'Token', 'Tag', 'ner_tags'],
    num_rows: 55183
})

In [21]:
example = train_df.loc[train_df["Record Number"] == 1, ["Record Number", "Token", "ner_tags"]].to_dict(orient="list")
example

{'Record Number': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'Token': ['Supreme',
  'Nike',
  'SB',
  'Dunk',
  'High',
  'By',
  'any',
  'Means',
  'Red',
  'US10',
  'EU44',
  'Supreme',
  'Box',
  'Logo',
  'Air',
  'Force'],
 'ner_tags': [20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 68, 68, 2, 24, 58]}

In [22]:
datasets.Dataset.from_dict(example)["ner_tags"]

[20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 68, 68, 2, 24, 58]

In [23]:
def combine(x):
  d = x.to_dict(orient="list")
  d["id"] = d["Record Number"][0]
  del d["Record Number"]
  del d["Title"]
  del d["Tag"]
  d["tokens"] = d.pop("Token")
  return d["ner_tags"], d["tokens"]

In [24]:
pre_ds = pd.DataFrame(train_df.groupby(["Record Number", "Title"]).apply(combine))

In [25]:
pre_ds

,,0
Record Number,Title,
1,Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force,"([20, 18, 24, 58, 25, 20, 54, 54, 10, 31, 8, 6..."
2,New Balance 530 Männer und Frauen Laufschuhe mit Buchstaben N bequeme Laufschuhe,"([18, 52, 20, 0, 68, 0, 23, 68, 21, 55, 68, 23..."
3,♥ MICHAEL KORS Sneaker Gr 39 ♥,"([68, 18, 52, 27, 68, 8, 68], [♥, MICHAEL, KOR..."
4,New Balance ML 574 EGO Turnschuhe grün,"([18, 52, 20, 54, 54, 23, 10], [New, Balance, ..."
5,"Nike air jordan 9 og , space jam , hare , bugs bunny , EU 45 / US 11 NEU + RECHNUNG","([18, 24, 58, 20, 54, 68, 10, 20, 68, 10, 68, ..."
...,...,...
4996,Adidas Raf Simons Sneakers,"([18, 24, 58, 27], [Adidas, Raf, Simons, Sneak..."
4997,"Ludwig Reiter Andalusier , Nubuk navy Schuhe Damen Gr . 39 NEU","([18, 52, 20, 68, 22, 10, 23, 0, 68, 68, 8, 68..."
4998,Adidas Superstar Laceless / Adidas Originals / Adidas Sneaker / weiß / schwarz / FV3017,"([18, 24, 32, 68, 18, 24, 68, 18, 27, 68, 10, ..."


In [26]:
pre_ds[['ner_tags', 'tokens']] = pd.DataFrame(pre_ds[0].tolist(), index=pre_ds.index)
pre_ds = pre_ds.reset_index().loc[:, ["Record Number", "tokens", "ner_tags"]].rename(columns={"Record Number":"id"})

In [27]:
ebay_ds = datasets.Dataset.from_pandas(pre_ds)
ebay_ds

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 5000
})

However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [28]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = lowercased_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [29]:
tokenized_ebay = ebay_ds.map(tokenize_and_align_labels, batched=True)
tokenized_ebay

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [30]:
data_collator = DataCollatorForTokenClassification(tokenizer=lowercased_tokenizer)

## Evaluate


In [31]:
seqeval = evaluate.load("seqeval")

In [32]:
label_list = list(label2id.keys())
len(label_list)

70

In [33]:


# labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [ ]:
tokenized_ebay.set_format("torch")

In [ ]:
tokenized_ebay_split = tokenized_ebay.train_test_split(0.2, shuffle=True)
tokenized_ebay_train = tokenized_ebay_split["train"]
tokenized_ebay_valid = tokenized_ebay_split["test"]

In [ ]:
# train on small set for now (at 4000, 1000 split this is doing nothing)
small_train_dataset = tokenized_ebay_train.shuffle(seed=42).select(range(4000)) #was previously 1000
small_eval_dataset = tokenized_ebay_valid.shuffle(seed=42).select(range(1000))

In [ ]:


train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [ ]:

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=70, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:



optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6934731665457822186
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14410383360
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14675505225237559987
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ebay_train,
    eval_dataset=tokenized_ebay_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.783357,0.780155,0.736473,0.757685,0.807433
2,1.124000,0.524572,0.850828,0.833909,0.842283,0.872290
3,1.124000,0.438876,0.873842,0.859027,0.866371,0.891966
4,0.387700,0.397954,0.886089,0.875981,0.881006,0.903625
5,0.387700,0.379501,0.895353,0.889168,0.892250,0.911733
6,0.242800,0.376421,0.892681,0.889691,0.891184,0.912097
7,0.242800,0.364992,0.901368,0.896180,0.898767,0.917471
8,0.171400,0.370484,0.897839,0.895866,0.896852,0.916925
9,0.171400,0.371778,0.901701,0.898587,0.900142,0.918929
10,0.129500,0.373066,0.902086,0.900576,0.901330,0.919293


TrainOutput(global_step=3750, training_loss=0.30510193481445314, metrics={'train_runtime': 366.9005, 'train_samples_per_second': 163.532, 'train_steps_per_second': 10.221, 'total_flos': 494524996344576.0, 'train_loss': 0.30510193481445314, 'epoch': 15.0})

<Tip>

For a more in-depth example of how to finetune a model for token classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification-tf.ipynb).

</Tip>

In [ ]:
torch.save(model.state_dict(), data_drive+"/model_lower")

## Load model (skip)

In [34]:
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=70, id2label=id2label, label2id=label2id
)
load_name = "/models/model_lower"
model.load_state_dict(torch.load(data_drive + load_name))
model.eval()
model.to("cuda")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

## Inference

In [ ]:
#ideas for improvement - some kind of Autoregression; previous token predicted class impacts next token?
# all of this data is for shoes.

In [35]:
sample = test_records.loc[test_records["Record Number"] == "6454", "Title"].values[0]
sample

'Minu Damen Schuhe Schnürschuhe 207536 Leder schwarz EUR 42'

In [36]:
inputs = lowercased_tokenizer(sample, return_tensors="pt")
cuda_in = inputs.input_ids.to("cuda")
decoded = lowercased_tokenizer.convert_ids_to_tokens(cuda_in[0])

with torch.no_grad():
    logits = model(cuda_in).logits
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
dict(zip(decoded, predicted_token_class))

{'[CLS]': 'No Tag',
 'min': 'Marke',
 '##u': 'I-Marke',
 'dame': 'Abteilung',
 '##n': 'No Tag',
 'sc': 'I-Produktart',
 '##hu': 'I-Produktart',
 '##he': 'No Tag',
 '##hn': 'I-Produktart',
 '##urs': 'I-Produktart',
 '##chu': 'I-Produktart',
 '207': 'Herstellernummer',
 '##53': 'I-Herstellernummer',
 '##6': 'I-Herstellernummer',
 'led': 'Obermaterial',
 '##er': 'No Tag',
 'schwarz': 'Farbe',
 'eu': 'EU-Schuhgröße',
 '##r': 'I-EU-Schuhgröße',
 '42': 'I-EU-Schuhgröße',
 '[SEP]': 'I-Abteilung'}

In [37]:
def similarity_score(word1, word2):
    # Calculate the Levenshtein distance
    levenshtein_distance = distance.levenshtein(word1, word2)

    # Calculate the similarity score (normalized)
    max_length = max(len(word1), len(word2))
    similarity = 1 - (levenshtein_distance / max_length)

    return similarity

In [38]:
def get_most_similar_word(text, word):
    #problem: EUR 42,5 when word = "eur 42 , 5"

    if " " in word:
        #check1: try to see by subtoken
        subtoken1 = word.split(" ")[0] #first subtoken in sequence
        start_index = text.find(get_most_similar_word(text, subtoken1))
        subtoken2 = word.split(" ")[-1] #last subtoken
        end_sim_word = get_most_similar_word(text, subtoken2)
        end_index = text.find(end_sim_word) + len(end_sim_word) #go till the end
        if start_index != -1 and start_index < end_index: #if not, just get most similar word
            return text[start_index:end_index]

    #check2: check a space-separated word
    words = pd.Series(text.split(" "))
    scores1 = words.apply(lambda x: similarity_score(x, word)) #score if lowercase word
    scores2 = words.apply(lambda x: similarity_score(x, word.upper())) #score if uppercase word
    scores = pd.concat([scores1, scores2],axis=1).max(axis=1)
    return words[scores.argmax()]

In [39]:
def case_outputs(outputs, text):
    lower_text = text.lower()

    for i in range(len(outputs)):
        token = outputs[i][1]
        subtoken = token.split(" ")[0] #consider only the first item in a multi-token
        start_index = lower_text.find(subtoken)
        last_subtoken = token.split(" ")[-1]
        end_index = lower_text.find(last_subtoken) + len(last_subtoken)
        cased_token = text[start_index:end_index]


        if start_index == -1 or start_index > end_index or cased_token == "": #not found
            outputs[i][1] = get_most_similar_word(text, token)
            continue

        outputs[i][1] = cased_token

    return outputs

In [40]:
#function to obtain tag predictions and merge aspect values
def process_title(title):
    inputs = lowercased_tokenizer(title, return_tensors="pt")
    cuda_in = inputs.input_ids.to("cuda")
    decoded = lowercased_tokenizer.convert_ids_to_tokens(cuda_in[0])

    model.eval()
    with torch.no_grad():
        logits = model(cuda_in).logits
    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

    # Merge I-<Tag> with <Tag>
    #Need to implement: checking previous token was the same tag, majority voting, ##
    merged_tokens = [] # list of [tag, token sequence]
    prev_tag, prev_token = "", ""
    current_tag = None

    for token, tag in zip(decoded, predicted_token_class):
      # exclude CLS and SEP
      if token in ['[CLS]', '[SEP]']:
          continue

      if token.startswith("##"):
          if len(merged_tokens) > 0:
              merged_tokens[-1][1] += token[2:]
              continue  # doing no majority voting, go to the next word
          else:
              token = token[2:]  # first observed token, act as if this subword is a complete word

      if tag.startswith('I-'):
          tag = tag[2:]
          # Check if the current "I-" tag matches the previous entry's tag
          if current_tag is not None and tag == current_tag:
              merged_tokens[-1][1] += ' ' + token
          else:
              merged_tokens.append([tag, token])
          current_tag = tag
          continue

      if tag != 'O' and token not in ['[CLS]', '[SEP]']:
          current_tag = tag
          merged_tokens.append([current_tag, token])
      else:
          current_tag = None

    # print(merged_tokens)
    return case_outputs(merged_tokens, title)


In [264]:
sample

'Minu Damen Schuhe Schnürschuhe 207536 Leder schwarz EUR 42'

In [41]:
process_title(sample)

[['Marke', 'Minu'],
 ['Abteilung', 'Damen'],
 ['Produktart', 'Schuhe Schnürschuhe'],
 ['Herstellernummer', '207536'],
 ['Obermaterial', 'Leder'],
 ['Farbe', 'schwarz'],
 ['EU-Schuhgröße', 'EUR 42']]

## Do prediction on test dataset


In [42]:
result_list = []
count = 0
for index, row in test_records.iterrows():
    if count % 1000 == 0:
      print(count)
    record_number = row['Record Number']
    title = row['Title']
    tagged_tokens = process_title(title)

    for tag, token_sequence in tagged_tokens:
        result_list.append({
            'Record Number': record_number,
            'Aspect Name': tag,
            'Aspect Value': token_sequence
        })
    count += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [43]:
result = pd.DataFrame(result_list)
# result = result.loc[(result["Aspect Name"] != "No Tag"), :] # remove No Tag, but NOT obscure
result["Record Number"] = result["Record Number"].astype(int)

In [44]:
result.tail()

,Record Number,Aspect Name,Aspect Value
250032,30000,Farbe,weiß
250033,30000,Abteilung,unisex
250034,30000,EU-Schuhgröße,43
250035,30000,No Tag,wie
250036,30000,No Tag,neu


In [45]:
test_records.loc[test_records["Record Number"] == "11817", "Title"].values

array(["Guess Women 's Shoes null Fashion Sneakers , Black , Size -1.0 bPOD US"],
      dtype=object)

In [63]:
result.loc[result["Record Number"] == 11817,:]

,Record Number,Aspect Name,Aspect Value
68080,11817,Marke,Guess
68081,11817,Abteilung,Women '
68082,11817,Marke,s
68083,11817,Produktart,Shoes
68084,11817,Modell,null
68085,11817,Obscure,Fashion
68086,11817,Stil,Sneakers
68087,11817,Obscure,","
68088,11817,Farbe,Black
68089,11817,Obscure,","


In [50]:
# check to make sure no NaN values
np.sum(result.isna())

Record Number    0
Aspect Name      0
Aspect Value     0
dtype: int64

In [54]:
result.loc[result["Aspect Name"]=="No Tag", :]

,Record Number,Aspect Name,Aspect Value
5,5001,No Tag,GR
6,5001,No Tag,.
17,5002,No Tag,NEU
21,5003,No Tag,Gr
22,5003,No Tag,.
...,...,...,...
250021,29998,No Tag,&
250022,29998,No Tag,Ungetragen
250023,29998,No Tag,Neu
250035,30000,No Tag,wie


In [53]:
result.loc[result["Aspect Value"]=="null", :] #still values to fix, but let's test for now
# CHECK NA REP: 11233, 11817

,Record Number,Aspect Name,Aspect Value
62193,11233,Modell,null
68084,11817,Modell,null


In [55]:
out = result.loc[result["Aspect Value"]!="null", :]
# out = out.loc[out["Aspect Value"] != "Obscure", :]
out

,Record Number,Aspect Name,Aspect Value
0,5001,Marke,NIKE
1,5001,Produktlinie,FREE
2,5001,Modell,RUN 3 SHIELD 5.0
3,5001,Stil,SNEAKERS
4,5001,Produktart,LAUFSCHUHE
...,...,...,...
250032,30000,Farbe,weiß
250033,30000,Abteilung,unisex
250034,30000,EU-Schuhgröße,43
250035,30000,No Tag,wie


In [56]:
# save to file
filename = "results_1122_casing_all.tsv"
out.to_csv(data_drive + "/results/" + filename, sep="\t", na_rep = "null", header=False, index=False)

## Tests (skip)

In [ ]:
#testing the I-tag fix

text = train_df.Title[0]
inputs = tokenizer(text, return_tensors="pt")
cuda_in = inputs.input_ids.to("cuda")
decoded = tokenizer.convert_ids_to_tokens(cuda_in[0])
model.eval() #not sure if this is correct
with torch.no_grad():
    logits = model(cuda_in).logits
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
dict(zip(decoded, predicted_token_class))


merged_tokens = [] # list of [tag, token sequence]
prev_tag, prev_token = "", ""
current_tag = None

for token, tag in zip(decoded, predicted_token_class):
    if token.startswith("##"):
        if len(merged_tokens) > 0:
            merged_tokens[-1][1] += token[2:]
            continue  # doing no majority voting, go to the next word
        else:
            token = token[2:]  # first observed token, act as if this subword is a complete word

    if tag.startswith('I-'):
        tag = tag[2:]
        # Check if the current "I-" tag matches the previous entry's tag
        if current_tag is not None and tag == current_tag:
            merged_tokens[-1][1] += ' ' + token
        else:
            merged_tokens.append([tag, token])
        current_tag = tag
        continue

    if tag != 'O' and token not in ['[CLS]', '[SEP]']:
        current_tag = tag
        merged_tokens.append([current_tag, token])
    else:
        current_tag = None

print(merged_tokens)


In [ ]:
ex_title = train_df.loc[0, "Title"]
ex_title

In [ ]:
tokenizer(ex_title)

In [ ]:
merged_tokens = [] # list of [tag, token sequence]
prev_tag, prev_token = "", ""
current_tag = None

for token, tag in zip(decoded, predicted_token_class):
    if token.startswith("##"):
        if len(merged_tokens) > 0:
            merged_tokens[-1][1] += token[2:]
            continue  # doing no majority voting, go to the next word
        else:
            token = token[2:]  # first observed token, act as if this subword is a complete word

    if tag.startswith('I-'):
        tag = tag[2:]
        # Check if the current "I-" tag matches the previous entry's tag
        if current_tag is not None and tag == current_tag:
            merged_tokens[-1][1] += ' ' + token
        else:
            merged_tokens.append([tag, token])
        current_tag = tag
        continue

    if tag != 'O' and token not in ['[CLS]', '[SEP]']:
        current_tag = tag
        merged_tokens.append([current_tag, token])
    else:
        current_tag = None

print(merged_tokens)
